In [1]:
# thread-safe print
from threading import Thread, Lock

lock = Lock()
def Print(*args):
    with lock:
        print(*args)
        
Print("hi")

hi


In [2]:
# topic creation
from kafka import KafkaAdminClient, KafkaProducer
from kafka.admin import NewTopic
broker = "kafka:9092"
admin = KafkaAdminClient(bootstrap_servers=[broker])
admin.create_topics([NewTopic("animals", 4, 1)])

CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='animals', error_code=0, error_message=None)])

# Producer

In [4]:
import string, random, threading, json, time
string.ascii_uppercase[:10]

'ABCDEFGHIJ'

In [7]:
from animals_pb2 import *

In [8]:
# producer
def animal_gen():
    Print("generating animals")
    producer = KafkaProducer(bootstrap_servers=[broker])
    while True:
        beach = random.choice(list(string.ascii_uppercase[:10]))
        animal = random.choice(["dolphin", "shark", "seagull"])
        s = Sighting(beach=beach, animal=animal)
        value = s.SerializeToString()
        key = bytes(beach, "utf-8")
        producer.send("animals", value=value, key=key)
        time.sleep(1)
threading.Thread(target=animal_gen).start()

generating animals


# Python Consumer

## Streaming Group By on Beach

In [12]:
from kafka import KafkaConsumer, TopicPartition

def beach_counter(partitions=[]):
    consumer = KafkaConsumer(bootstrap_servers=[broker])
    consumer.assign([TopicPartition("animals", part_num) for part_num in partitions])
    consumer.seek_to_beginning()
    
    counts = {}
    
    #TODO: while True:
    for i in range(10):
        batch = consumer.poll(1000)
        for tp, messages in batch.items():
            for msg in messages:
                s = Sighting.FromString(msg.value)
                if s.beach not in counts:
                    counts[s.beach] = 0
                counts[s.beach] += 1
        Print(counts)
        
beach_counter([0,1,2,3])


{'J': 60, 'A': 81, 'F': 68, 'I': 75, 'B': 77, 'C': 61, 'D': 44, 'E': 34}
{'J': 60, 'A': 81, 'F': 68, 'I': 75, 'B': 77, 'C': 61, 'D': 79, 'E': 69, 'H': 81, 'G': 66}
{'J': 60, 'A': 82, 'F': 68, 'I': 75, 'B': 77, 'C': 61, 'D': 79, 'E': 69, 'H': 81, 'G': 66}
{'J': 60, 'A': 82, 'F': 68, 'I': 75, 'B': 77, 'C': 61, 'D': 80, 'E': 69, 'H': 81, 'G': 66}
{'J': 60, 'A': 83, 'F': 68, 'I': 75, 'B': 77, 'C': 61, 'D': 80, 'E': 69, 'H': 81, 'G': 66}
{'J': 60, 'A': 83, 'F': 68, 'I': 75, 'B': 77, 'C': 61, 'D': 80, 'E': 69, 'H': 81, 'G': 66}
{'J': 60, 'A': 83, 'F': 68, 'I': 75, 'B': 77, 'C': 61, 'D': 81, 'E': 69, 'H': 81, 'G': 66}
{'J': 60, 'A': 83, 'F': 68, 'I': 75, 'B': 77, 'C': 61, 'D': 81, 'E': 69, 'H': 81, 'G': 66}
{'J': 60, 'A': 83, 'F': 68, 'I': 76, 'B': 77, 'C': 61, 'D': 81, 'E': 69, 'H': 81, 'G': 66}
{'J': 60, 'A': 84, 'F': 68, 'I': 76, 'B': 77, 'C': 61, 'D': 81, 'E': 69, 'H': 81, 'G': 66}


## Stream Group By on Animal

# Spark Consumer